Vediamo il caso di cilindro immerso in un flusso 

In [ ]:
import gmsh
import os
import numpy as np
import matplotlib.pyplot as plt
import tqdm.autonotebook
from mpi4py import MPI
from petsc4py import PETSc
from basix.ufl import element
from dolfinx.cpp.mesh import to_type, cell_entity_type
from dolfinx.fem import (Constant, Function, functionspace,assemble_scalar, dirichletbc, form, locate_dofs_topological, set_bc)
from dolfinx.fem.petsc import (apply_lifting, assemble_matrix, assemble_vector,
                               create_vector, create_matrix, set_bc)
from dolfinx.graph import adjacencylist
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells
from dolfinx.io import (VTXWriter, distribute_entity_data, gmshio)
from dolfinx.mesh import create_mesh, meshtags_from_entities
from ufl import (FacetNormal, Identity, Measure, TestFunction, TrialFunction,
                 as_vector, div, dot, ds, dx, inner, lhs, grad, nabla_grad, rhs, sym, system)

from tqdm.notebook import tqdm
from tqdm.autonotebook import tqdm

from xCFL_number import CFLnumber

gmsh.initialize()  #Serve per inizializzare l'ambiente di lavoro Gmsh! Dev'essere fatto per forza prima di qualsiasi altra chiamata Gmsh!!!!!
L = 2.2
H = 1 # In realtà nella mesh originale H=0.41
#coordinate del centro del cilindro
c_x = 0.2 # In realtà nella mesh originale c_x=0.2
c_y = 0.5 # In realtà nella mesh originale c_y=0.2
r = 0.05

gdim = 2
mesh_comm = MPI.COMM_WORLD
model_rank = 0
if mesh_comm.rank == model_rank:   #Serve per far sì che la geometria venga creata su di un solo processo (cioè quello con rank=model_rank, cioè di regola 0)
#vado ad aggiungere un rettangolo e l'ostacolo nella OpenCASCADE CAD representation
    rectangle = gmsh.model.occ.addRectangle(0, 0, 0, L, H, tag=1)
    obstacle = gmsh.model.occ.addDisk(c_x, c_y, 0, r, r)
#Le 2 righe di codice appena viste non generano nessuna mesh!!! Semplicemente stanno descrivendo geometricamente il dominio 

In [ ]:
#vado poi a sottrare il disco dalla geometria rettangolare 
#in maniera tale che non dovremo in seguito meshare anche l'interno del cerchio 

if mesh_comm.rank == model_rank:
    fluid = gmsh.model.occ.cut([(gdim, rectangle)], [(gdim, obstacle)])

    #Adesso che quindi ho definito geometricamente il dominio vado a sincronizzarlo su tutti i processi
    gmsh.model.occ.synchronize()
    #Da adesso in poi quindi la geometria sarà disponibile al motore di meshatura di Gmsh!!!

-Vado a definire un marker (per taggare) del gruppo fisico (2D) del fluido 

(mi servirà per poi successivamente "meshare" il fluido)

In [ ]:
#To get GMSH to mesh the fluid, we add a physical volume marker

fluid_marker = 1
if mesh_comm.rank == model_rank:
    volumes = gmsh.model.getEntities(dim=gdim)
    #in questo caso particolare avrò che: volumes=[(2,1)]
    #2 è la dimensione della superficie (perchè stiamo considerando un dominio rettangolare "bucato" dal disco)
    #1 è il tags dell'entità (è stato assegnato direttamente da Gmsh)

    assert (len(volumes) == 1) #serve semplicemente per fare una verifica che nel dominio ci sia un'unica entità geometrica 2D

    gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], fluid_marker) #Definisco quindi il gruppo fisico
    #volumes[0][0] <---è uguale a 2 (ed indica la dimensione dell'entità considerata)
    #[volumes[0][1]] <--è un array con unico elemento 1 (che sarà il tags identificativo dell'entità)

    gmsh.model.setPhysicalName(volumes[0][0], fluid_marker, "Fluid") #assegno un nome al gruppo fisico!!!

Adesso vado invece a "taggare" le varie superfici della mesh, cioé:

2) Inflow (x=0,y)
3) Outflow (x=L,y)
4) Fluid walls, cioè i top (x,y=H) e bottom (x,y=0) boundaries 
5) Pareti dell'ostacolo (disco)

We will do this by computing the center of mass for each geometrical entity.

In [ ]:
#definisco i vari marker 
inlet_marker, outlet_marker, wall_marker, obstacle_marker = 2, 3, 4, 5
inflow, outflow, walls, obstacle = [], [], [], []

if mesh_comm.rank == model_rank:
    boundaries = gmsh.model.getBoundary(volumes, oriented=False)
    #boundaries sarà quindi una lista di tuple(dim,tag) realtive ad ogni boundary di volumes
    
    for boundary in boundaries:
        center_of_mass = gmsh.model.occ.getCenterOfMass(boundary[0], boundary[1])
        #boundary[0]-->sarà la dim del boundary in considerazione 
        #boundary[1]-->sarà il tag del boundary in considerazione
        #getCenterOfMass-->restituisce quindi le coordinate (x,y,z) del "centro" del boundary in considerazione
        if np.allclose(center_of_mass, [0, H / 2, 0]):
            inflow.append(boundary[1]) #quindi assegno ad "inflow" (che avevo creato precedentemente) il tag relativo al boundary che gli corrisponde!
        elif np.allclose(center_of_mass, [L, H / 2, 0]):
            outflow.append(boundary[1])
        elif np.allclose(center_of_mass, [L / 2, H, 0]) or np.allclose(center_of_mass, [L / 2, 0, 0]):
            walls.append(boundary[1])
        else:
            obstacle.append(boundary[1])
    #quindi, grazie a questo ciclo for, sono riuscito ad individuare i vari dati dei vari boundaries del dominio 2D (volumes)


    gmsh.model.addPhysicalGroup(1, walls, wall_marker)
    gmsh.model.setPhysicalName(1, wall_marker, "Walls")

    gmsh.model.addPhysicalGroup(1, inflow, inlet_marker)
    gmsh.model.setPhysicalName(1, inlet_marker, "Inlet")

    gmsh.model.addPhysicalGroup(1, outflow, outlet_marker)
    gmsh.model.setPhysicalName(1, outlet_marker, "Outlet")
    
    gmsh.model.addPhysicalGroup(1, obstacle, obstacle_marker)
    gmsh.model.setPhysicalName(1, obstacle_marker, "Obstacle")

In questo problema ci converrà utilizzare una mesh unstructured!

In particolare, data la natura del problema ci convverrà avere una mesh più densa vicino all'ostacolo!!
(e meno densa lontana da esso)

In [ ]:
# Create distance field from obstacle.
# Add threshold of mesh sizes based on the distance field
# LcMax -                  /--------
#                      /
# LcMin -o---------/
#        |         |       |
#       Point    DistMin DistMax

res_min = r / 3   #r=0.05 è il raggio del disco
if mesh_comm.rank == model_rank:
    distance_field = gmsh.model.mesh.field.add("Distance") #vado a creare un nuovo campo di mesh size a cui assegnerò il fieldTyp: 'Distance'
    #distance field conterrà il tag di questo nuovo campo di mesh size  
    
    gmsh.model.mesh.field.setNumbers(distance_field, "EdgesList", obstacle)
    threshold_field = gmsh.model.mesh.field.add("Threshold")

    gmsh.model.mesh.field.setNumber(threshold_field, "IField", distance_field)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMin", res_min)
    gmsh.model.mesh.field.setNumber(threshold_field, "LcMax", 0.25 * 0.41) # H=0.41
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMin", r)
    gmsh.model.mesh.field.setNumber(threshold_field, "DistMax", 2 * 0.41) # H=0.41
    min_field = gmsh.model.mesh.field.add("Min")
    gmsh.model.mesh.field.setNumbers(min_field, "FieldsList", [threshold_field])
    gmsh.model.mesh.field.setAsBackgroundMesh(min_field)


Andiamo quindi a meshare 
In this demo, to match the DFG 2D-3 benchmark, we use second order quadrilateral elements.

In [ ]:
if mesh_comm.rank == model_rank:
    gmsh.option.setNumber("Mesh.Algorithm", 8)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)
    
    gmsh.model.mesh.generate(gdim)
    gmsh.model.mesh.setOrder(2)
    gmsh.model.mesh.optimize("Netgen")

As we have generated the mesh, we now need to load the mesh and corresponding facet markers into DOLFINx. 

In [ ]:
mesh, _, ft = gmshio.model_to_mesh(gmsh.model, mesh_comm, model_rank, gdim=gdim)
ft.name = "Facet markers"

# DEFINING THE MOST IMPORTANT VARIABLES

In [ ]:
dt=0.01  # nel problema del Poiseulle flow avevo: dt=0.000625
t=0

Re=80 # Oppure 150
nu=1/Re
restart=False
tol=1e-5 # oppure 1e-8 o addirittura 1e-15?
tot_save= 5000 # we save every -tot_save- steps

step=0
step_init = 2000 # 2 mila (step a cui si inizia ad calcolare u_mean, p_mean ed f)
stop_to_restart= 120 # step a cui stoppare il codice e salvafre tutto!! (per poi restartare)

step_final = 100000000 # 100 milion

# DEFINIZIONE DEL PROBLEMA VARIAZIONALE 

Adesso che il dominio è stato definito posso andare a specificare i vari parametri del problema

In [ ]:
from dolfinx import fem
#We want to use a continuous piecewise quadratic elements for the velocity and continuous piecewise linear elements for the pressure.
v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
V = fem.functionspace(mesh, v_cg2)
Q = fem.functionspace(mesh, s_cg1)

In [ ]:
from ufl import  TestFunction, TrialFunction

# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)

p = TrialFunction(Q)
q = TestFunction(Q)

from dolfinx.fem import Function
# Create useful functions
u0 = Function(V) 
u0.name="u0"
u1 = Function(V)
u1.name='u1'

p0=Function(Q)
p0.name="p0"
p1 = Function(Q)
p1.name="p1"

In [ ]:
#definisco le funzioni che mi serviranno durante il calcolo di mean flow e forzaggio 
u_mean_old = Function(V)
u_mean = Function(V)
u_mean.name= "u_mean"
u_fluct = Function(V)
u_fluct.name="u_fluct"

f_old = Function(V)
f= Function(V)
f.name="forcing"
f_save = Function(V)

p_mean = Function(Q)

In [ ]:
from dolfinx.fem import Function, dirichletbc, locate_dofs_topological
from ufl import div, dx, inner, lhs, rhs
from dolfinx import default_scalar_type

In [ ]:
#Dirichelet values  

#velocity
u_inlet=PETSc.ScalarType((1.0,0.0)) 
u_nonslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType) 
u_y_top_bottom=PETSc.ScalarType(0.0) 

#pressure 
p_in=PETSc.ScalarType(8.0) 
p_out=PETSc.ScalarType(0) # provare ad usare 1e-6??? In modo tale da non avere una pressione nulla?!

In [ ]:
fdim = mesh.topology.dim - 1

# BC for the velocity 
bcu_inflow = dirichletbc(u_inlet, locate_dofs_topological(V, fdim, ft.find(inlet_marker)),V)
bcu_walls = dirichletbc(u_y_top_bottom, locate_dofs_topological(V.sub(1), fdim, ft.find(wall_marker)), V.sub(1))
bcu_obstacle = dirichletbc(u_nonslip, locate_dofs_topological(V, fdim, ft.find(obstacle_marker)), V)
bcu = [bcu_inflow, bcu_obstacle, bcu_walls]

# BC for the pressure
bcp_outlet = dirichletbc(p_out, locate_dofs_topological(Q, fdim, ft.find(outlet_marker)), Q)
bcp = [bcp_outlet] 

# LINEAR PROBLEMS
Let's define the 3 linear problem (Chorin. method)

As a further verification of our implementation, we compute the drag (C_D) and lift coefficients (C_L) over the obstacle

In [ ]:
from ufl import inner, nabla_grad, dot, grad, ds, dx, lhs, rhs, div 
from dolfinx.fem import form 
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc


k=fem.Constant(mesh,PETSc.ScalarType(dt)) # time-step
f_zero= fem.Constant(mesh, PETSc.ScalarType((0, 0))) # this should be used only at the very first step!!!

F1=(1/k)*inner(u-u0,v)*dx 
F1+=inner(grad(u0) * u0, v) * dx # inner(dot(u0,nabla_grad(u0)),v)*dx
F1+=nu*inner(grad(u),grad(v))*dx
F1-=inner(f_zero,v)*dx

#definisco quindi la classica forma: a(u,v)=L(v)
a1=form(lhs(F1))
L1=form(rhs(F1))

A1 = assemble_matrix(a1, bcs=bcu)
A1.assemble()
b1 = create_vector(L1)

In [ ]:
#Step 2 
#caluclation of the new pressure (p_n) (by using u*)
a2=inner(grad(p),grad(q))*dx 
L2=-(1/k)*div(u1)*q*dx #non si usa inner o dot (poiché sia div(u1) che q sono degli scalari!)

a2=form(a2)
L2=form(L2)

A2 = assemble_matrix(a2, bcp)
A2.assemble()
b2 = create_vector(L2)

In [ ]:
#Step 3 
#calculation of the new velocity (u_n) (by using u* and p_n)
a3=inner(u,v)*dx
L3=inner(u1,v)*dx - k*inner(grad(p1),v)*dx

a3=form(a3)
L3=form(L3)


A3= assemble_matrix(a3, bcs=bcu) # Applico comunque le BC
# A3= assemble_matrix(a3) 
# Ad A3 (e neanche a b3) non c'é bisogno di applicare le BC poiché queste erano già state applicate nei primi 2 step 
# quindi, implicitamente verranno rispettate anche dal risultato dello step 3  
A3.assemble()
b3= create_vector(L3)

In [ ]:
# Solver for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

# DRAG AND LIFT CALCULATION

In [ ]:
n = -FacetNormal(mesh)  # Normal pointing out of obstacle
dObs = Measure("ds", domain=mesh, subdomain_data=ft, subdomain_id=obstacle_marker)
u_t = inner(as_vector((n[1], -n[0])), u1)
rho = Constant(mesh, PETSc.ScalarType(1))

drag = form(2 / 1 * (nu * rho * inner(grad(u_t), n) * n[1] - p1 * n[0]) * dObs)
lift = form(-2 / 1 * (nu * rho * inner(grad(u_t), n) * n[0] + p1 * n[1]) * dObs)

DRAG=[]
LIFT=[]

# VTXWriter to save the variables

In [ ]:
from pathlib import Path
import os
from dolfinx.io import XDMFFile, VTXWriter


folder = Path("results_V4")
folder.mkdir(exist_ok=True, parents=True)

# Apro i file XDMF in modalità append
if os.path.exists(folder / "velocity_u.xdmf"):
    xdmf_u = XDMFFile(mesh.comm, folder / "velocity_u.xdmf", "a") # lo metto solo in modalità append (quindi evito di sovrascrivere la mesh)
else:
    xdmf_u = XDMFFile(mesh.comm, folder / "velocity_u.xdmf", "a")
    xdmf_u.write_mesh(mesh)

vtx_u = VTXWriter(mesh.comm, folder/"velocity_u.bp", [u1, u_mean, u_fluct, f], engine="BP4")
vtx_p = VTXWriter(mesh.comm, folder/"pressure_p.bp", [p1], engine="BP4") 

if restart:
    with XDMFFile(mesh.comm, folder / "velocity_u.xdmf", "r") as xdmf_in:
        #xdmf_in.read_mesh(mesh)  # necessario solo se la mesh non è già costruita

        # Leggi l'ultima istanza della funzione
        xdmf_in.read_function(u1, step=-1)  # step=-1 => ultimo step temporale
        u0.x.array[:] = u1.x.array[:]
        xdmf_in.read_function(u_mean, step=-1)
        xdmf_in.read_function(u_fluct, step=-1)
        xdmf_in.read_function(f, step=-1)

        # individuo il valore di t 
        time_steps = xdmf_in.time_steps()
        t = time_steps[-1]  

        #individuo anche quanti step sono stati fatti 
        step = len(time_steps)
        print(f'######### RESTARTING AT STEP: {step} ##########')

# SOLVING THE VARIATIONAL PROBLEM

As we are solving a time dependent problem with many time steps, we use the tqdm-package to visualize the progress.

In [ ]:
#progress = tqdm.autonotebook.tqdm(desc="Solving PDE", total=num_steps)
progress = tqdm(desc="Solving PDE", total=step_final)

In [ ]:
from dolfinx.fem import apply_lifting, assemble_scalar
from dolfinx.fem.petsc import LinearProblem

while step <=step_final:  #al posto di num_steps (in realtà dovrebbe esserci step_final!!!!!!)
    progress.update(1)

    # Step 1: Tentative velocity step
    with b1.localForm() as loc_1:
        loc_1.set(0)
    assemble_vector(b1, L1)
    apply_lifting(b1, [a1], [bcu])
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b1, bcu)
    solver1.solve(b1, u1.x.petsc_vec)
    u1.x.scatter_forward()

    # Step 2: Pressure corrrection step
    with b2.localForm() as loc_2:
        loc_2.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp)
    solver2.solve(b2, p1.x.petsc_vec)
    p1.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc_3:
        loc_3.set(0)
    assemble_vector(b3, L3)
    apply_lifting(b3, [a3], [bcu])
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b3, bcu)
    solver3.solve(b3, u1.x.petsc_vec)
    u1.x.scatter_forward()


    if step > step_init:
        actual_step=step-step_init
        
        u_mean.x.array[:]=(actual_step/(actual_step+1))*u_mean.x.array[:] + 1/(actual_step+1)*u1.x.array[:]
        p_mean.x.array[:]=(actual_step/(actual_step+1))*p_mean.x.array[:] + 1/(actual_step+1)*p1.x.array[:]

        u_fluct.x.array[:]=u1.x.array[:]-u_mean.x.array[:]

        ####### Calcolo del forcing f ########
        # voglio calcolare f, che risulta uguale a: f_prod= -dot(grad(u_fluct), u_fluct)
        expr = dot(grad(u_fluct), u_fluct)
        # Problema variazionale per proiettare l'espressione sul dominio 
        prod = TrialFunction(V)
        v_f = TestFunction(V)
        a_f = inner(prod, v_f) * dx
        L_f = inner(expr, v_f) * dx
        # Risolvi la proiezione L2
        problem_f = LinearProblem(a_f, L_f, bcs=[], petsc_options={"ksp_type": "cg"})
        f_prod = problem_f.solve()  
        f_prod.x.array[:] *= -1 
        
        f.x.array[:]=(actual_step/(actual_step+1))*f_old.x.array[:] + 1/(actual_step+1)*f_prod.x.array[:]

        
        #calcolo la differenza (norma) fra le variabili calcolate (mean_flow e forcing) in 2 iterazioni successive
        # mi serve per poter verificare la convergenza  
        err_mean = np.linalg.norm((u_mean_old.x.array[:] - u_mean.x.array[:])) / np.linalg.norm(u_mean_old.x.array[:])
        err_forc = np.linalg.norm((f_old.x.array[:] - f.x.array[:])) / np.linalg.norm(f_old.x.array[:])


        # si potrebbe impostare una tolleranza più tranquilla!! Come ad esempio 1e-5 oppure 1e-8
        if (err_mean < tol and err_forc < tol) or step == step_final:
            cfl_max = CFLnumber(mesh, u1, dt, V)
            MPI.COMM_WORLD.Barrier()

            if step == step_final:
                print(f'Last step reached: L2 mean:{err_mean}, L2 forc:{err_forc}')
                #print(f'Last step reached: L2 mean:{err_mean}, L2 forc:{err_forc}, CFLmax: {cfl_max}')
            else:
                print(f'Convergence reached: L2 mean:{err_mean}, L2 forc:{err_forc}')
                #print(f'Convergence reached: L2 mean:{err_mean}, L2 forc:{err_forc}, CFLmax: {cfl_max}')

            # Prima di interromprere il codice poiché siamo andati a convergenza 
            # salvo tutte le variabili di mio interesse 

            vtx_u.write(t)
            vtx_p.write(t)
                      
            DRAG.append(assemble_scalar(drag))
            LIFT.append(assemble_scalar(lift))
            # Printo l'evoluzione dei Cd e Cl calcolati 
            print('')
            print(f'''Drag evolution: 
                    First calculation, Drag: {DRAG[0]}
                    Mid-way calculation, Drag: {DRAG[len(DRAG)//2]}
                    Final calculation, Drag: {DRAG[-1]}''')
            
            print(f'''Lift evolution: 
                    First calculation, Lift: {LIFT[0]}
                    Mid-way calculation, Lift: {LIFT[len(LIFT)//2]}
                    Final calculation, Lift: {LIFT[-1]}''')

            MPI.COMM_WORLD.Barrier()
            # break mi farà uscire dal ciclo for originale!! (for i in range(num_steps))
            break

        if step % tot_save==0:
            MPI.COMM_WORLD.Barrier()
            #salvo "a prescindere" la soluzione ogni tot steps


            vtx_u.write(t)
            vtx_p.write(t)

            DRAG.append(assemble_scalar(drag))
            LIFT.append(assemble_scalar(lift))

    
    # Write solutions to file (in realtà non lo dovrei fare sempre!!!)
    # vtx_u.write(t)
    # vtx_p.write(t)
    
    if step==1:
            #salvo i primi valori di Drag e Lift calcolati
            DRAG.append(assemble_scalar(drag))
            LIFT.append(assemble_scalar(lift))


    ######################## Let's move to the next step!!!!! ########################
    # Update variable with solution form this time step
    u0.x.array[:] = u1.x.array[:] # aggiorno il valore della u0 (che rappresenta u(n-1) nello step 1 dell'IPCS) 
    u_mean_old.x.array[:]=u_mean.x.array[:]

    f_old.x.array[:]=f.x.array[:] # aggiono il valore del forzaggio 
    # p0.x.array[:] = p1.x.array[:] # aggiorno il valore della p0


    t += dt    
    step+=1
    # # Compute error at current time-step
    # error_L2 = np.sqrt(mesh.comm.allreduce(assemble_scalar(L2_error), op=MPI.SUM))
    # error_max = mesh.comm.allreduce(np.max(u_.x.petsc_vec.array - u_ex.x.petsc_vec.array), op=MPI.MAX)
    # # Print error only every 20th step and at the last step
    # if (i % 20 == 0) or (i == num_steps - 1):
    #     print(f"Time {t:.2f}, L2-error {error_L2:.2e}, Max error {error_max:.2e}")


# Close xmdf file
xdmf_u.close()
vtx_u.close()
vtx_p.close()

b1.destroy()
b2.destroy()
b3.destroy()
solver1.destroy()
solver2.destroy()
solver3.destroy()